In [4]:
import math
import array
import wave
import tempfile
import os
from mutagen.mp4 import MP4
from pydub import AudioSegment

def generate_sine_wave(frequency, duration, sample_rate=44100, amplitude=32767):
    num_samples = int(sample_rate * duration)
    samples = array.array('h', [int(amplitude * math.sin(2 * math.pi * frequency * t / sample_rate))
                                for t in range(num_samples)])
    return samples

def create_wav_file(filename, samples, sample_rate=44100):
    with wave.open(filename, 'w') as wav_file:
        wav_file.setnchannels(1)  # Mono
        wav_file.setsampwidth(2)  # 2 bytes per sample
        wav_file.setframerate(sample_rate)
        wav_file.writeframes(samples.tobytes())

def create_m4a_with_timed_lyrics(audio_segments, phrases, output_file):
    # Concatenate audio segments
    combined_audio = AudioSegment.empty()
    current_time = 0
    timed_lyrics = []

    for segment, phrase in zip(audio_segments, phrases):
        # Load the audio segment
        audio = AudioSegment.from_wav(segment)
        
        # Add to the combined audio
        combined_audio += audio
        
        # Calculate timestamp
        minutes, seconds = divmod(current_time / 1000, 60)
        timestamp = f"[{int(minutes):02d}:{seconds:05.2f}]"
        
        # Add to timed lyrics
        timed_lyrics.append(f"{timestamp}{phrase}")
        
        # Update current time
        current_time += len(audio)

    # Export combined audio to M4A
    temp_m4a = "temp.m4a"
    combined_audio.export(temp_m4a, format="ipod")

    # Add lyrics to the M4A file
    audio = MP4(temp_m4a)
    
    # Join all lyrics into a single string
    lyrics_text = "\n".join(timed_lyrics)
    
    # Add lyrics using the standard lyrics tag
    audio["\xa9lyr"] = lyrics_text
    
    audio.save()

    # Rename the temp file to the desired output name
    os.replace(temp_m4a, output_file)

# Generate example sine waves
frequencies = [440, 880, 1320]  # A4, A5, E6
duration = 3  # 3 seconds each
sample_rate = 44100

temp_files = []
for i, freq in enumerate(frequencies):
    samples = generate_sine_wave(freq, duration, sample_rate)
    temp_file = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    create_wav_file(temp_file.name, samples, sample_rate)
    temp_files.append(temp_file.name)

# Example usage
audio_segments = temp_files
phrases = [
    f"Tone lots of text lots of text lots of text lots of text {i+1}: {freq} Hz" for i, freq in enumerate(frequencies)
]
output_file = "output_timed_lyrics_long_text.m4a"

create_m4a_with_timed_lyrics(audio_segments, phrases, output_file)

print(f"Created {output_file} with timed lyrics.")

# Clean up temporary files
for temp_file in temp_files:
    os.remove(temp_file)

Created output_timed_lyrics_long_text.m4a with timed lyrics.
